# Bokeh Code for Goodreads Plots

## Import Pandas

In [1]:
import pandas as pd

## Import necessary Bokeh modules

In [2]:
from bokeh.io import output_notebook, show
from bokeh.layouts import layout, Spacer, column, row
from bokeh.models.widgets import AutocompleteInput
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models.callbacks import CustomJS
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh import events
from bokeh.models import Range1d, NumeralTickFormatter, ColorBar
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.palettes import Spectral6

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
goodreads_df = pd.read_csv('Classics-Plot-Information.csv')

In [5]:
goodreads_df.head()

,title,author,numRatings,whichListCategory,yearFirstPublished,averageRating,color,fill_alpha,line_color,line_alpha,line_width
0,The Handmaid's Tale,Margaret Atwood,1181201,Most Shelved and Most Read Classic,1985,4.10,#d53e4f,0.7,black,0,0
1,Animal Farm,George Orwell,2407884,Most Shelved and Most Read Classic,1945,3.92,#d53e4f,0.7,black,0,0
2,The Alchemist,Paulo Coelho,1808108,Most Read Classic,1988,3.85,#fee08b,0.7,black,0,0
3,1984,George Orwell,2678662,Most Shelved and Most Read Classic,1949,4.17,#d53e4f,0.7,black,0,0
4,Fahrenheit 451,Ray Bradbury,1459387,Most Shelved and Most Read Classic,1953,3.98,#d53e4f,0.7,black,0,0


## Added styling directly into CSV

In [11]:
# def add_color(row):
#     if row['whichListCategory'] == 'Most Read':
#         return '#fee08b'
#     elif row['whichListCategory'] == 'Most Shelved':
#         return '#3288bd'
#     elif row['whichListCategory'] == 'Most Shelved and Most Read':
#         return '#d53e4f'

In [16]:
# goodreads_df['color'] = goodreads_df.apply(add_color, axis=1)
# goodreads_df['fill_alpha'] = .7
# goodreads_df['line_color'] = 'black'
# goodreads_df['line_alpha'] = 0
# goodreads_df['line_width'] = 0

## Upload HTML Template

In [51]:
from jinja2 import Environment, FileSystemLoader

env = Environment(
    loader=FileSystemLoader('.')
)
template = env.get_template('simple_bokeh_viz_template.html')

## Set Source

In [52]:
source = ColumnDataSource(goodreads_df)

# Make Goodreads Plot By Category

In [53]:
TOOLTIPS = [
    ("Title", "@title"),
    ("Author", "@author"),
    ("Total # of Ratings", "@numRatings{(0.00a)}"),
    ("Classics Category", "@whichListCategory"),
    ("Publication Date", "@yearFirstPublished")
    
]

bokeh_plot = figure(plot_width=1000, plot_height=800,tooltips=TOOLTIPS, tools=" pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
         x_axis_label = 'Publication Date', y_axis_label ='Number of Goodreads Ratings', x_range=Range1d(1500, 2050, bounds=(0, 2050))
                    ,y_range =Range1d(-1000, 4300000, bounds=(-100000, 4500000)))



color_map = factor_cmap(field_name='whichListCategory',
                 palette=['#fee08b', '#3288bd','#d53e4f'], factors=['Most Read', 'Most Shelved', 'Most Shelved and Most Read'])


bokeh_plot.circle(x='yearFirstPublished', 
         y='numRatings',
         size = 40,
         source=source, fill_alpha='fill_alpha',
         color='color', line_color='line_color', line_width='line_width', line_alpha= 'line_alpha',
        hover_alpha=1, hover_line_color='black', hover_line_width=2, legend_group = 'whichListCategory')


autocomp = AutocompleteInput(completions=goodreads_df.title.tolist(), title='Search by title')
author_autocomp = AutocompleteInput(completions=list(set(goodreads_df.author.tolist())), title='Search by author')


callback = CustomJS(args=dict(source=source), code="""
    var titles = source.data.title;
    var autocomplete = cb_obj.value;
    var alpha = source.data.fill_alpha;
    var line_width = source.data.line_width
    var line_color = source.data.line_color
    var line_alpha = source.data.line_alpha

    if (autocomplete === '') {
        for (var i = 0; i < alpha.length; i++) {
             alpha[i] = .7;
             line_width[i] = 0;
            line_color[i] = 'none';
            line_alpha[i] = 0;
            reset_btn = document.getElementsByClassName('bk-tool-icon-reset')
            reset_btn[0].click()
        }  
    } else {
        for (i = 0; i < titles.length; i++) {
            if (titles[i] === autocomplete) {
                alpha[i] = 1.0;
                line_width[i] = 4.0;
                line_color[i] = 'black';
                line_alpha[i] = 1;
            } else {
                alpha[i] = 0.3;
                line_width[i] = 0;
                line_alpha[i] = 0;
                 line_color[i] = 'none';
        
            }
        }
    }
    
            
    source.change.emit();
    console.log(autocomplete);
""")


author_callback = CustomJS(args=dict(source=source), code="""


    var authors = source.data.author;
    var author_autocomplete = cb_obj.value;
    var alpha = source.data.fill_alpha;
    var line_width = source.data.line_width
    var line_color = source.data.line_color
    var line_alpha = source.data.line_alpha

    if (author_autocomplete === '') {
        for (var i = 0; i < authors.length; i++) {
             alpha[i] = .7;
             line_width[i] = 0;
              line_color[i] = 'none';
            line_alpha[i] = 0;
        }  
    } else {
        for (i = 0; i < authors.length; i++) {
            if (authors[i] === author_autocomplete) {
                alpha[i] = 1.0;
                line_width[i] = 4.0;
                line_color[i] = 'black';
                line_alpha[i] = 1;
            } else {
                alpha[i] = 0.3;
                line_width[i] = 0;
                line_alpha[i] = 0;
                 line_color[i] = 'none';
            }
        }
    }
            
    source.change.emit();
""")

tap_callback = CustomJS(args=dict(source=source),code="""
    var titles = source.data.title;
    var alpha = source.data.fill_alpha;
    var line_width = source.data.line_width
    var line_color = source.data.line_color
    var line_alpha = source.data.line_alpha
    
    for (var i = 0; i < titles.length; i++) {
            alpha[i] = .7;
            line_width[i] = 0;
            line_color[i] = 'none';
            line_alpha[i] = 0;}
    source.change.emit();
    console.log("CLICK!");
""")



# execute a callback whenever the plot canvas is tapped
bokeh_plot.js_on_event(events.Tap, tap_callback)

         
#callback = CustomJS(args={'source':source}, code=code)
autocomp.js_on_change('value', callback)
author_autocomp.js_on_change('value', author_callback)

# Set Titles and Labels
bokeh_plot.title.text_font_size = '40pt'
bokeh_plot.xaxis.axis_label_text_font_size = "20pt"
bokeh_plot.title.text_font = "Helvetica"
bokeh_plot.xaxis.axis_label_text_font_style = "normal"
bokeh_plot.yaxis.axis_label_text_font_size = "20pt"
bokeh_plot.yaxis.axis_label_text_font_style = "normal"
bokeh_plot.yaxis.major_label_text_font_size = "15pt"
bokeh_plot.xaxis.major_label_text_font_size = "15pt"

bokeh_plot.legend.label_text_font_size = '15pt'


# Set Legend
bokeh_plot.legend.location = "top_left"

#Format big numbers with abbreviations
bokeh_plot.yaxis.formatter = NumeralTickFormatter(format='0.0a')

search_boxes = row(Spacer(width=250), autocomp, author_autocomp)
full_layout = column(bokeh_plot, Spacer(height=20),search_boxes)


show(full_layout)
html = file_html(full_layout, CDN, "Goodreads-Plot", template=template)

## Write to HTML File

In [54]:
with open('Goodreads-Classics-Plot.html', 'w') as outfile:
    outfile.write(html)

## Upload HTML Template

In [55]:
from jinja2 import Environment, FileSystemLoader

env = Environment(
    loader=FileSystemLoader('.')
)
template = env.get_template('simple_bokeh_viz_template.html')

## Set Source

In [56]:
source = ColumnDataSource(goodreads_df)

# Make Goodreads Plot By Rating

In [63]:
TOOLTIPS = [
    ("Title", "@title"),
    ("Author", "@author"),
     ("Average Goodreads Ratings", "@averageRating{(0.0)}"),
    ("Classics Category", "@whichListCategory"),
    ("Publication Date", "@yearFirstPublished"),
     ("Total # of Ratings", "@numRatings{(0.00a)}")
    
]

bokeh_plot = figure(plot_width=1000, plot_height=750,tooltips=TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
         x_axis_label = 'Publication Date', y_axis_label ='Number of Goodreads Ratings', x_range=Range1d(1500, 2050, bounds=(0, 2050))
                    ,y_range =Range1d(-1000, 4300000, bounds=(-100000, 4500000)))


color_map = linear_cmap(field_name='averageRating', palette=Spectral6, low=2.9, high=4.6)


bokeh_plot.circle(x='yearFirstPublished', 
         y='numRatings',
         size = 40,
         source=source, fill_alpha='fill_alpha',
         color=color_map, line_color='line_color', line_width='line_width', line_alpha= 'line_alpha',
        hover_alpha=1, hover_line_color='black', hover_line_width=2 )


autocomp = AutocompleteInput(completions=goodreads_df.title.tolist(), title='Search by title')
author_autocomp = AutocompleteInput(completions=list(set(goodreads_df.author.tolist())), title='Search by author')


callback = CustomJS(args=dict(source=source), code="""
    var titles = source.data.title;
    var autocomplete = cb_obj.value;
    var alpha = source.data.fill_alpha;
    var line_width = source.data.line_width
    var line_color = source.data.line_color
    var line_alpha = source.data.line_alpha

    if (autocomplete === '') {
        for (var i = 0; i < alpha.length; i++) {
             alpha[i] = .7;
             line_width[i] = 0;
            line_color[i] = 'none';
            line_alpha[i] = 0;
        }  
    } else {
        for (i = 0; i < titles.length; i++) {
            if (titles[i] === autocomplete) {
                alpha[i] = 1.0;
                line_width[i] = 4.0;
                line_color[i] = 'black';
                line_alpha[i] = 1;
            } else {
                alpha[i] = 0.5;
                line_width[i] = 0;
                line_alpha[i] = 0;
                 line_color[i] = 'none';
        
            }
        }
    }
    
            
    source.change.emit();
    console.log(autocomplete);
""")


author_callback = CustomJS(args=dict(source=source), code="""


    var authors = source.data.author;
    var author_autocomplete = cb_obj.value;
    var alpha = source.data.fill_alpha;
    var line_width = source.data.line_width
    var line_color = source.data.line_color
    var line_alpha = source.data.line_alpha

    if (author_autocomplete === '') {
        for (var i = 0; i < authors.length; i++) {
             alpha[i] = .7;
             line_width[i] = 0;
              line_color[i] = 'none';
            line_alpha[i] = 0;
        }  
    } else {
        for (i = 0; i < authors.length; i++) {
            if (authors[i] === author_autocomplete) {
                alpha[i] = 1.0;
                line_width[i] = 4.0;
                line_color[i] = 'black';
                line_alpha[i] = 1;
            } else {
                alpha[i] = 0.5;
                line_width[i] = 0;
                line_alpha[i] = 0;
                 line_color[i] = 'none';
            }
        }
    }
            
    source.change.emit();
""")

tap_callback = CustomJS(args=dict(source=source),code="""
    var titles = source.data.title;
    var alpha = source.data.fill_alpha;
    var line_width = source.data.line_width
    var line_color = source.data.line_color
    var line_alpha = source.data.line_alpha
    
    for (var i = 0; i < titles.length; i++) {
            alpha[i] = .7;
            line_width[i] = 0;
            line_color[i] = 'none';
            line_alpha[i] = 0;}
    source.change.emit();
    console.log("CLICK!");
""")

color_bar = ColorBar(color_mapper=color_map['transform'], width=10,  location=(0,0))
bokeh_plot.add_layout(color_bar, 'right')

# execute a callback whenever the plot canvas is tapped
bokeh_plot.js_on_event(events.Tap, tap_callback)

         
#callback = CustomJS(args={'source':source}, code=code)
autocomp.js_on_change('value', callback)
author_autocomp.js_on_change('value', author_callback)

# Set Titles and Labels
bokeh_plot.title.text_font_size = '40pt'
bokeh_plot.xaxis.axis_label_text_font_size = "20pt"
bokeh_plot.title.text_font = "Helvetica"
bokeh_plot.xaxis.axis_label_text_font_style = "normal"
bokeh_plot.yaxis.axis_label_text_font_size = "20pt"
bokeh_plot.yaxis.axis_label_text_font_style = "normal"
bokeh_plot.yaxis.major_label_text_font_size = "15pt"
bokeh_plot.xaxis.major_label_text_font_size = "15pt"



#Format big numbers with abbreviations
bokeh_plot.yaxis.formatter = NumeralTickFormatter(format='0.0a')

search_boxes = row(Spacer(width=250), autocomp, author_autocomp)
full_layout = column(bokeh_plot, Spacer(height=20),search_boxes)


show(full_layout)
html = file_html(full_layout, CDN, "Goodreads-Plot", template=template )

## Write to HTML File

In [64]:
with open('Goodreads-Classics-Ratings.html', 'w') as outfile:
    outfile.write(html)